In [ ]:
import pandas as pd
import torch
from peft import LoraConfig, TaskType, get_peft_model
from torch.utils.data import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
)

In [ ]:
def load_model(
    model_name: str = "meta-llama/Meta-Llama-3-8B-Instruct", load_on_gpu: bool = True
) -> tuple[AutoModelForCausalLM, AutoTokenizer]:
    if load_on_gpu:
        config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=False,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
        )

        model = AutoModelForCausalLM.from_pretrained(
            model_name, low_cpu_mem_usage=True, quantization_config=config
        )
    else:
        model = AutoModelForCausalLM.from_pretrained(model_name, low_cpu_mem_usage=True)

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.pad_token_id = tokenizer.eos_token_id

    return model, tokenizer

In [ ]:
# Load the model & tokenizer
model, tokenizer = load_model()

In [ ]:
# Add Lora Matrices
lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

peft_model = get_peft_model(model, lora_config)

In [ ]:
def tokenize_queries(query: str, max_length: int = 1024) -> torch.Tensor:
    query = [
        {"role": "system", "content": "You are a recipe generator. Give the necessary ingredients and steps to create the input dish."},
        {"role": "user", "content": query},
    ]

    input_ids = tokenizer.apply_chat_template(
        query,
        padding='max_length', 
        max_length=max_length,
        add_generation_prompt=True,
        return_tensors="pt"
    )

    return input_ids

def tokenize_responses(response: str, max_length: int = 1024) -> torch.Tensor:
    response = [
        {"role": "assistant", "content": response},
    ]

    labels = tokenizer.apply_chat_template(
        response,
        padding='max_length', 
        max_length=max_length,
        add_generation_prompt=True,
        return_tensors="pt"
    )

    return labels

In [ ]:
# Load in the data
train_data = pd.read_csv("./data/train.csv")
test_data = pd.read_csv("./data/test.csv")

In [ ]:
# Tokenize the data
train_data["input_ids"] = train_data.apply(lambda row: tokenize_queries(row["query"]), axis=1)
train_data["labels"] = train_data.apply(lambda row: tokenize_responses(row["response"]), axis=1)

test_data["input_ids"] = test_data.apply(lambda row: tokenize_queries(row["query"]), axis=1)
test_data["labels"] = test_data.apply(lambda row: tokenize_responses(row["response"]), axis=1)

# Drop unecessary columns
train_data = train_data.drop(columns="query").drop(columns="response")
test_data = test_data.drop(columns="query").drop(columns="response")

In [ ]:
class RecipeDataset(Dataset):
    def __init__(self, data: pd.DataFrame):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            "input_ids": self.data.iloc[idx]["input_ids"][0],
            "labels": self.data.iloc[idx]["labels"][0],
        }

In [ ]:
# Setup trainer
training_args = TrainingArguments(
    output_dir="./output",
    # auto_find_batch_size=True,
    per_device_train_batch_size=2,
    learning_rate=1e-3,
    num_train_epochs=1,
    logging_steps=1,
    max_steps=1,
    report_to="wandb"
)

trainer = Trainer(model=peft_model, 
                      args=training_args,
                      train_dataset=RecipeDataset(train_data),
                      eval_dataset=RecipeDataset(test_data)
        )

In [ ]:
# Now train
trainer.train()